Api endpoints



In [21]:
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
import csv
import time

# Setup Selenium WebDriver
service = Service(r'C:\Users\sshak\Downloads\chromedriver-win64\chromedriver-win64\chromedriver.exe')  # Update the path
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

base_url = "https://www.freepik.com/search?ai=excluded&format=search&last_filter=query&last_value=indian+people&query=indian+people&selection=1&type=photo"

# CSV file setup
csv_file_path = "dynamic_links_with_download_urls.csv"
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Index", "Extracted Numbers", "Download URL"])  # Write header

try:
    # Open the first page
    driver.get(base_url)
    time.sleep(3)  # Adjust delay as needed

    index = 1  # Link index counter

    while True:
        # Extract links with the specific class on the current page
        specific_class = "_1286nb1h _1286nb1k _1286nb14ku _1286nb14m6 _1286nb14ni _1286nb14ou _1286nb196"
        links_with_specific_class = driver.find_elements(By.XPATH, f'//a[@class="{specific_class}"]')

        # Extract href attributes
        dynamic_links = [
            link.get_attribute('href')
            for link in links_with_specific_class
            if link.get_attribute('href')
        ]

        # Extract only the number before '.htm' using regular expression
        for link in dynamic_links:
            # Use regex to extract the number after the last underscore and before '.htm'
            match = re.search(r'_(\d+)\.htm', link)
            if match:
                extracted_number = match.group(1)  # Extract the matched number
                # Construct the download URL
                download_url = f"https://www.freepik.com/api/regular/download?resource={extracted_number}&size=2000&action=copy"
                
                # Write to CSV
                with open(csv_file_path, mode='a', newline='', encoding='utf-8') as file:
                    writer = csv.writer(file)
                    writer.writerow([index,download_url])
                    index += 1

        print(f"Extracted {len(dynamic_links)} links from the current page.")

        # Check for the "Next Page" link and navigate
        try:
            next_button = driver.find_element(By.XPATH, '//a[@data-cy="pagination-next"]')  # Based on provided HTML
            next_page_url = next_button.get_attribute("href")
            print(f"Navigating to the next page: {next_page_url}")
            driver.get(next_page_url)
            time.sleep(3)  # Adjust delay for page loading
        except NoSuchElementException:
            print("No more pages to scrape. Exiting.")
            break
finally:
    # Close the WebDriver
    driver.quit()


C:\Users\sshak\AppData\Local\Temp\ipykernel_6276\305754663.py:38: DeprecationWarning: using WebElement.get_attribute() has been deprecated. Please use get_dom_attribute() instead.
  if link.get_attribute('href')
C:\Users\sshak\AppData\Local\Temp\ipykernel_6276\305754663.py:36: DeprecationWarning: using WebElement.get_attribute() has been deprecated. Please use get_dom_attribute() instead.
  link.get_attribute('href')


Extracted 50 links from the current page.
Navigating to the next page: https://www.freepik.com/search?ai=excluded&format=search&last_filter=page&last_value=2&page=2&query=indian+people&selection=1&type=photo#uuid=30db82b4-f531-4b84-8ed5-103108b7d9e5


C:\Users\sshak\AppData\Local\Temp\ipykernel_6276\305754663.py:61: DeprecationWarning: using WebElement.get_attribute() has been deprecated. Please use get_dom_attribute() instead.
  next_page_url = next_button.get_attribute("href")


KeyboardInterrupt: 

In [ ]:
import requests
import json
import csv
import time
with open('cookies.json', 'r') as file:
    cookies_list = json.load(file)

# Convert list of cookies into a dictionary
cookies_dict = {cookie['name']: cookie['value'] for cookie in cookies_list}

# Function to fetch and extract URL from API response with retry logic
def fetch_url_from_api(api_url, retries=3, delay=5):
    try:
        # Send a GET request to fetch the JSON response
        response = requests.get(api_url,cookies=cookies_dict)
        response.raise_for_status()  # Raise an error for HTTP issues

        # Parse the JSON content
        data = response.json()

        # Extract the "url" field
        extracted_url = data.get("url")
        if extracted_url:
            print(f"Extracted URL from {api_url}: {extracted_url}")
        else:
            print(f"No 'url' field found in the response from {api_url}.")

    except requests.exceptions.RequestException as e:
        if response.status_code == 429 and retries > 0:  # Handle rate limiting
            print(f"Rate limit exceeded for {api_url}. Retrying in {delay} seconds...")
            time.sleep(delay)  # Wait before retrying
            fetch_url_from_api(api_url, retries - 1, delay)  # Retry with one less attempt
        else:
            print(f"Error fetching data from {api_url}: {e}")

# Open the CSV file containing API endpoints
csv_file = 'dynamic_links_with_download_urls.csv'  # Replace with your CSV file path
with open(csv_file, newline='', encoding='utf-8') as csvfile:
    csvreader = csv.reader(csvfile)
    
    # Skip the header row if there is one
    next(csvreader, None)  # Skip header row, remove if no header
    
    # Loop through the rows in the CSV file
    for row in csvreader:
        if row:  # Ensure the row is not empty
            api_url = row[1]  # Assuming the URL is in the second column (index 1)
            fetch_url_from_api(api_url)


Extracted URL from https://www.freepik.com/api/regular/download?resource=10706637&size=2000&action=copy: https://downloadscdn7.freepik.com/1157/42/41030.jpg?filename=indian-man-city-male-traditional-turban-hinduist-summer-city&token=exp=1733802825~hmac=1e6c7e0c62f57b4c21b28134c137d815
Extracted URL from https://www.freepik.com/api/regular/download?resource=34304463&size=2000&action=copy: https://downloadscdn7.freepik.com/23/2149916/2149915905.jpg?filename=medium-shot-smiley-man-posing&token=exp=1733802826~hmac=29f9c74534f62e2c6997b3db75acb74f
Extracted URL from https://www.freepik.com/api/regular/download?resource=2765262&size=2000&action=copy: https://downloadscdn7.freepik.com/53876/15/14330.jpg?filename=indian-people-using-mobile-phone&token=exp=1733802827~hmac=e4f66bd54b63ffa623e9fdd18204fa62
Extracted URL from https://www.freepik.com/api/regular/download?resource=13013029&size=2000&action=copy: https://downloadscdn7.freepik.com/176474/45/44483.jpg?filename=young-woman-standing-stra

In [1]:
import json
import requests
import csv

# Load cookies from the cookies.json file
with open('cookies.json', 'r') as file:
    cookies_list = json.load(file)

# Convert list of cookies into a dictionary
cookies_dict = {cookie['name']: cookie['value'] for cookie in cookies_list}

# Open the CSV file containing the URLs
with open('dynamic_links_with_download_urls.csv', 'r') as file:
    reader = csv.reader(file)
    
    # Skip the header if there's one
    next(reader, None)  # Uncomment if there's a header row
    
    # Loop through each URL in the CSV
    for row in reader:
        url = row[1]  # Assuming the URL is in the first column
        
        try:
            # Make the GET request using requests and pass the cookies as a dictionary
            response = requests.get(url, cookies=cookies_dict)
            
            # Print the response content (or process it as needed)
            print(f"Response for {url}: {response.text}")
            # If you want to save the response content to a file
            # with open(f'response_{url.split("/")[-1]}.txt', 'w') as response_file:
            #     response_file.write(response.text)
        
        except requests.exceptions.RequestException as e:
            print(f"Error with {url}: {e}")



Response for https://www.freepik.com/api/regular/download?resource=10706637&size=2000&action=copy: {"filename":"41030.jpg","url":"https://downloadscdn7.freepik.com/1157/42/41030.jpg?filename=indian-man-city-male-traditional-turban-hinduist-summer-city&token=exp=1733764555~hmac=7aa04622f900a902556cc6b0085275fd","signedUrl":"https://img.freepik.com/free-photo/indian-man-city-male-traditional-turban-hinduist-summer-city_1157-41030.jpg?t=st=1733763655~exp=1733767255~hmac=e124019893a6412258cae9a67d3ae247a06b8f0cc2a325f0db296f38a9155d2a"}
Response for https://www.freepik.com/api/regular/download?resource=34304463&size=2000&action=copy: {"filename":"2149915905.jpg","url":"https://downloadscdn7.freepik.com/23/2149916/2149915905.jpg?filename=medium-shot-smiley-man-posing&token=exp=1733764557~hmac=b4846111a0e9d85380718acc6f7020ae","signedUrl":"https://img.freepik.com/free-photo/medium-shot-smiley-man-posing_23-2149915905.jpg?t=st=1733763657~exp=1733767257~hmac=b2da204c15ae2c39700efec7270f24af4a2

KeyboardInterrupt: 